In [ ]:
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

T_MAX = 300      # Анализируем первые 5 минут (300 секунд)
FPS_EXTRACT = 1  # Частота кадров (1 кадр/сек)
L_MIN, L_MAX = 5, 30  # Длительность заставки (сек)


In [ ]:
def extract_features(video_path: str) -> list:
    """Извлекает цветовые гистограммы из видео."""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    features = []
    frame_count = 0

    while frame_count < T_MAX * fps:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % int(fps) == 0:
            frame = cv2.resize(frame, (224, 224))
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hist = cv2.calcHist([hsv], [0, 1, 2], None, [16, 4, 4], [0, 180, 0, 256, 0, 256])
            hist = cv2.normalize(hist, None).flatten()
            features.append(hist)
        frame_count += 1
    cap.release()
    return features


In [ ]:

def find_intro(video_files: list) -> tuple:
    """Находит интервал заставки."""
    all_features = [extract_features(f) for f in video_files[:5]]
    min_len = min(len(f) for f in all_features)
    all_features = [f[:min_len] for f in all_features]

    d_t = np.zeros(min_len)
    for t in range(min_len):
        dists = []
        for i in range(len(all_features)):
            for j in range(i + 1, len(all_features)):
                dist = np.linalg.norm(all_features[i][t] - all_features[j][t])
                dists.append(dist)
        d_t[t] = np.mean(dists)

    best_score = float('inf')
    best_t, best_L = 0, 0

    for L in range(L_MIN, L_MAX + 1):
        for t in range(min_len - L):
            score = np.mean(d_t[t:t + L])
            if score < best_score:
                best_score = score
                best_t, best_L = t, L

    start, end = best_t, best_t + best_L - 1
    return start, end



In [ ]:
path = "/content/-220020068_456257141.mp4"
video_files = sorted(glob(path))
start, end = find_intro(video_files)
print(f"Заставка: {start}-{end} секунд")

Заставка: 0--1 секунд
